In [36]:
import tensorflow as tf
import numpy as np
import datetime

if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility with tensor flow 1.x")

%load_ext tensorboard
#%reload_ext tensorboard

Enabled compatitility with tensor flow 1.x
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [37]:
dataset = np.load('training_data.npy')

dataset[0:, 0] /= 1000

In [38]:
x = dataset[:,1].reshape(-1, 1)
y = dataset[:,0].reshape(-1, 1)

# Módelo Gradient Descent

In [39]:
def gradient_descent(x, y, epochs, lr, print_rate=100):
    tf.reset_default_graph()
    
    with tf.name_scope("input"):
        x = tf.concat([x, tf.ones_like(x)], axis = 1)
    with tf.name_scope("params"):
        params = tf.Variable(name="params", initial_value = tf.zeros((2, 1), tf.float64))
    with tf.name_scope("predict"):
        y_hat = tf.matmul(x, params)
    with tf.name_scope("error"):
        error = 1/2 * tf.reduce_mean(tf.math.square(y - y_hat))
    with tf.name_scope("gradient"):
        gradients = tf.gradients(error, params)
    with tf.name_scope("update_model"):
        fit = tf.scalar_mul(-lr, gradients[0])
        update_model = tf.assign(params, tf.add(params, fit))


    with tf.Session() as session:
        summary = tf.summary.scalar(name = "error_epochs", tensor = error)
        writer = tf.summary.FileWriter("./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_lr=" + str(lr) + "_epochs=" + str(epochs), session.graph)
        session.run(tf.global_variables_initializer())
        for epoch in range(0, epochs):
            if ((epoch + 1) % print_rate) == 0:
                print("error: " + str(session.run(error)))
      
            writer.add_summary(session.run(summary), epoch)
            session.run(update_model)

        params = session.run(params)

        writer.close()
    session.close()

    return params

## Grafo del módelo

![text](graphs/model.png)

Buscaremos un buen learning rate y epoch para nuestro módelo, dependiendo si empizan a diverger o que tan rápido disminuye el error iremos cambiando los parámetros a probar

In [40]:
def test_hyper_parmas(lrs, epochs=1000):
    print_rate = epochs / 5
    for lr in lrs:
        print("lr=" + str(lr))
        gradient_descent(x, y, epochs, lr, print_rate)

In [41]:
test_hyper_parmas([0.0001, 0.0005, 0.0009])

lr=0.0001
error: 5088.956758291787
error: 2162.742408269533
error: 1575.0080102649333
error: 1456.660140225012
error: 1432.5294785144506
lr=0.0005
error: 1432.3470514335775
error: 1424.2614773656267
error: 1421.9446667905006
error: 1419.6515940904962
error: 1417.3802969582684
lr=0.0009
error: 1424.7427929227767
error: 1420.5752834939058
error: 1416.4867205366022
error: 1412.467777671251
error: 1408.5172694029434


![text](graphs/000x.png)

Al parecer un learning rate más alto hace converger el error más rapido, entonces probaremos con valores mayores a los anteriores

In [42]:
test_hyper_parmas([0.001, 0.005, 0.009])

lr=0.001
error: 1424.2728454219214
error: 1419.6628965028528
error: 1415.141603446656
error: 1410.705772169294
error: 1406.353787266486
lr=0.005
error: 1406.437968313122
error: 1385.8717745435767
error: 1367.1773640259707
error: 1350.1843808557787
error: 1334.737973666289
lr=0.009
error: 1389.9060411224264
error: 1356.8491885187395
error: 1329.0098754317455
error: 1305.564589746283
error: 1285.819798660553


![text](graphs/00x.png)

Este caso parece similar al anterior, entonces probaremos con valores más grandes

In [43]:
test_hyper_parmas([0.01, 0.05, 0.09])

lr=0.01
error: 1385.965144830706
error: 1350.257662665267
error: 1320.7547398468257
error: 1296.3782674089744
error: 1276.2374694219363
lr=0.05
error: 67490.81733677507
error: 245155.79344382486
error: 899881.6757441885
error: 3312015.2022008067
error: 12198522.149722412
lr=0.09
error: 6.41440343111138e+169
error: inf
error: inf
error: nan
error: nan


Al parecer con valores mayores a 0.5 el error del módelo empiza a diverger, por lo que probaremos con valores entre 0.1 y 0.4

In [44]:
test_hyper_parmas([0.01, 0.02, 0.03, 0.04])

lr=0.01
error: 1385.965144830706
error: 1350.257662665267
error: 1320.7547398468257
error: 1296.3782674089744
error: 1276.2374694219363
lr=0.02
error: 1350.4043719133924
error: 1296.4678508176912
error: 1259.650292170172
error: 1234.5182926042671
error: 1217.3629616865508
lr=0.03
error: 1320.9845254064558
error: 1259.7042915902425
error: 1225.1486390011335
error: 1205.6628594683643
error: 1194.6749146445593
lr=0.04
error: 1296.6473439311007
error: 1234.582181464807
error: 1205.6731486973038
error: 1192.2077497143168
error: 1185.9357664477277


![text](graphs/0x.png)

In [45]:
test_hyper_parmas([0.041, 0.045, 0.049])

lr=0.041
error: 1294.4580369350901
error: 1232.5569242631477
error: 1204.2703316278119
error: 1191.3443713602164
error: 1185.4376702246857
lr=0.045
error: 1286.104701531549
error: 1225.1851942103697
error: 1199.397016379246
error: 1188.480478391776
error: 1183.8593375569203
lr=0.049
error: 1278.3680588506857
error: 1218.8555896826397
error: 1195.5207753152222
error: 1186.3702850213838
error: 1182.7820210949678


![text](graphs/04x.png)

Aunque al principio el error con 0.49 no disminuye tan rápido al final es el que más disminuye de los 3, así que nos quedaremos con 0.49.

In [46]:
test_hyper_parmas([0.049], 2000)

lr=0.049
error: 1218.8555896826397
error: 1186.3702850213838
error: 1181.3749229404254
error: 1180.6067712305526
error: 1180.4886502535705


In [57]:
test_hyper_parmas([0.049], 10000)

lr=0.049
error: 1180.4886502535705
error: 1180.4671876290395
error: 1180.4671857836718
error: 1180.467185783514
error: 1180.4671857835122


In [62]:
test_hyper_parmas([0.049], 4000)

lr=0.049
error: 1186.3702850213838
error: 1180.6067712305526
error: 1180.4704864390233
error: 1180.4672638312393
error: 1180.4671876290395


In [61]:
gradient_descent(x, y, 4000, 0.049, 200)

error: 1278.3680588506857
error: 1218.8555896826397
error: 1195.5207753152222
error: 1186.3702850213838
error: 1182.7820210949678
error: 1181.3749229404254
error: 1180.8231449059454
error: 1180.6067712305526
error: 1180.5219226782785
error: 1180.4886502535705
error: 1180.4756028388986
error: 1180.4704864390233
error: 1180.4684800991968
error: 1180.4676933351448
error: 1180.4673848142916
error: 1180.4672638312393
error: 1180.4672163890684
error: 1180.4671977851442
error: 1180.4671904898205
error: 1180.4671876290395


array([[ 45.43444887],
       [-96.19740908]])

### Conclusión

Se demostró que un learning rate de 0.49 es bastante rápido en converger al minimo error, aunque podríamos encontrar valores mejores entre 0.0491 y 0.049, pero considero que 0.49 es lo suficientemente bueno. Usando el learning rate de 0.49 el error llega rapidamente a 1180, y después de 4000 iteraciones el valor del error no baja de 1180.4671, por lo que considero que agregar más epochs no hará una gran diferencia y/o no valdría la pena.

Entonces los parametros del módelo quedarían en 45.2898101 y -95.27101687, con los hyper parametros 0.49 y 4000